In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

# statsmodels API 불러오기
import statsmodels.api as sm

# 분산 팽창 계수 함수 불러오기
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
# 교차검증 횟수
cv_count = 10

# 교차 검증
kfold = KFold(n_splits=cv_count, shuffle=True, random_state=1)

In [3]:
# 학습 데이터와 테스트 데이터 불러오기
train_df = pd.read_csv("train_final_with_pca.csv")
test_df  = pd.read_csv("test_final_with_pca.csv")

# ID 컬럼 제거
train_df = train_df.drop(columns=['ID'])

# X와 y 분리
X = train_df.drop(columns=['Segment'])
y = train_df['Segment']

# 라벨 인코딩
encoder = LabelEncoder()
y_num   = encoder.fit_transform(y)

# X는 숫자형이므로 바로 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [4]:
# 교차검증으로 성능 평가
f1_scores  = []
acc_scores = []

for train_idx, val_idx in kfold.split(X_scaled):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y_num[train_idx],   y_num[val_idx]
    
    model = XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        use_label_encoder=False,
        eval_metric='mlogloss'
    )
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    f1_scores.append(f1_score(y_val, y_pred, average='micro'))
    acc_scores.append(accuracy_score(y_val, y_pred))

In [5]:
# 교차검증 결과 출력
print(f'Cross-Validation Micro F1: {np.mean(f1_scores):}')
print(f'Cross-Validation Accuracy: {np.mean(acc_scores):}')

Cross-Validation Micro F1: 0.9061925000000001
Cross-Validation Accuracy: 0.9061925000000001


In [6]:
1/0

ZeroDivisionError: division by zero

### XGBoost

In [ ]:
# 전체 데이터로 재학습
final_model = XGBClassifier(
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    use_label_encoder=False,
    eval_metric='mlogloss'
)
final_model.fit(X_scaled, y_num)

In [ ]:
# 테스트 데이터 전처리
test_ids      = test_df['ID']
X_test        = test_df.drop(columns=['ID', 'Segment'], errors='ignore')
X_test_scaled = scaler.transform(X_test)

# 테스트 데이터 예측
pred_num = final_model.predict(X_test_scaled)
pred_str = encoder.inverse_transform(pred_num)

In [ ]:
# 예측 결과를 담은 DataFrame 생성
submission = pd.DataFrame({'ID': test_ids, 'Segment': pred_str})

In [ ]:
# 샘플 제출 파일 불러오기
sample = pd.read_csv('open/sample_submission.csv')

In [ ]:
# 샘플 파일의 Segment 컬럼을 예측 결과로 대체
sample['Segment'] = sample['ID'].map(submission.set_index('ID')['Segment'])

sample

In [ ]:
# 최종 제출파일 저장
sample.to_csv('9조_최종모델.csv', index=False, encoding='utf-8-sig')